In [1]:
class State:
    def __init__(self, position, path):
        self.position = position
        self.path = path

    def goalTest(self, goal):
        return self.position == goal

    def moveGen(self, grid):
        n = len(grid)
        x, y = self.position
        moves = []
        for dx, dy in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n and grid[nx][ny] == 0:
                moves.append(State((nx, ny), self.path + [(nx, ny)]))
        return moves


def heuristic(a, b):
    dx = abs(a[0] - b[0])
    dy = abs(a[1] - b[1])
    return max(dx, dy)

# Best First Search (Greedy)
def best_first_search(grid):
    import heapq
    from itertools import count
    n = len(grid)
    start, goal = (0, 0), (n - 1, n - 1)

    if grid[0][0] == 1 or grid[n-1][n-1] == 1:
        return -1, []

    visited = set()
    start_state = State(start, [start])
    counter = count()
    pq = [(heuristic(start, goal), next(counter), start_state)]  

    while pq:
        _, __, current_state = heapq.heappop(pq)

        if current_state.goalTest(goal):
            return len(current_state.path), current_state.path

        if current_state.position in visited:
            continue
        visited.add(current_state.position)

        for neighbor in current_state.moveGen(grid):
            if neighbor.position not in visited:
                h = heuristic(neighbor.position, goal)
                heapq.heappush(pq, (h, next(counter), neighbor))

    return -1, []

# A* Search
def a_star_search(grid):
    import heapq
    from itertools import count
    n = len(grid)
    start, goal = (0, 0), (n - 1, n - 1)

    if grid[0][0] == 1 or grid[n-1][n-1] == 1:
        return -1, []

    start_state = State(start, [start])
    counter = count()
    open_set = [(heuristic(start, goal), next(counter), 0, start_state)]  # (f, tie, g, state)
    g_scores = {start: 0}

    while open_set:
        _, __, g, current_state = heapq.heappop(open_set)

        if current_state.goalTest(goal):
            return len(current_state.path), current_state.path

        for neighbor in current_state.moveGen(grid):
            tentative_g = g + 1
            pos = neighbor.position
            if pos not in g_scores or tentative_g < g_scores[pos]:
                g_scores[pos] = tentative_g
                f = tentative_g + heuristic(pos, goal)
                heapq.heappush(open_set, (f, next(counter), tentative_g, neighbor))

    return -1, []

# Example Test Cases
if __name__ == "__main__":
    test_grids = [
        [[0, 1], [1, 0]],
        [[0, 0, 0], [1, 1, 0], [1, 1, 0]],
        [[1, 0, 0], [1, 1, 0], [1, 1, 0]]
    ]

    for i, grid in enumerate(test_grids, 1):
        bfs_len, bfs_path = best_first_search(grid)
        astar_len, astar_path = a_star_search(grid)
        print(f"Example {i}:")
        print(f"Best First Search  → Path length: {bfs_len}, Path: {bfs_path if bfs_len != -1 else []}")
        print(f"A* Search          → Path length: {astar_len}, Path: {astar_path if astar_len != -1 else []}")
        print()


Example 1:
Best First Search  → Path length: 2, Path: [(0, 0), (1, 1)]
A* Search          → Path length: 2, Path: [(0, 0), (1, 1)]

Example 2:
Best First Search  → Path length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]
A* Search          → Path length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]

Example 3:
Best First Search  → Path length: -1, Path: []
A* Search          → Path length: -1, Path: []

